# Clustering Analysis

In [ ]:
pkgs <- sort(c(
'tidyverse',
'factoextra',
'dbscan',
'cluster',
'mclust',
'kernlab',
'e1071',
'scatterpie',
'fpc',
'seriation',
'mlbench',
'GGally'
))
  
lapply(pkgs, function(pkg) {
  if (system.file(package = pkg) == '') install.packages(pkg)
})

all_pkgs <- union(all_pkgs, pkgs)

**Packages used for this chapter:** `r format_pkgs(pkgs)`

You can read the free sample chapter from the textbook [@Tan2005]:
[Chapter 7. Cluster Analysis: Basic Concepts and
Algorithms](https://www-users.cs.umn.edu/~kumar001/dmbook/ch7_clustering.pdf)

## Data Preparation

In [ ]:
library(tidyverse)

We will use here a small and very clean dataset called Ruspini which is
included in the R package **cluster**.

In [ ]:
data(ruspini, package = "cluster")

The Ruspini data set, consisting of 75 points in four groups that is
popular for illustrating clustering techniques. It is a very simple data
set with well separated clusters. The original dataset has the points
ordered by group. We can shuffle the data (rows) using `sample_frac`
which samples by default 100%.

In [ ]:
ruspini <- as_tibble(ruspini) %>% sample_frac()
ruspini

### Data cleaning

In [ ]:
ggplot(ruspini, aes(x = x, y = y)) + geom_point()

summary(ruspini)

For most clustering algorithms it is necessary to handle missing values
and outliers (e.g., remove the observations). For details see Section
"Outlier removal" below. This data set has not missing values or strong
outlier and looks like it has some very clear groups.

### Scale data

Clustering algorithms use distances and the variables with the largest
number range will dominate distance calculation. The summary above shows
that this is not an issue for the Ruspini dataset with both, x and y,
being roughly between 0 and 150. Most data analysts will still scale
each column in the data to zero mean and unit standard deviation
(z-scores). *Note:* The standard `scale()` function scales a whole data
matrix so we implement a function for a single vector and apply it to
all numeric columns.

In [ ]:
## I use this till tidyverse implements a scale function
scale_numeric <- function(x) x %>% mutate_if(is.numeric, function(y) as.vector(scale(y)))

ruspini_scaled <- ruspini %>% scale_numeric()
summary(ruspini_scaled)

After scaling, most z-scores will fall in the range $[-3,3]$ (z-scores
are measured in standard deviations from the mean), where $0$ means
average.

## Clustering methods

### k-means Clustering

[k-means](https://en.wikipedia.org/wiki/K-means_clustering) implicitly
assumes Euclidean distances. We use $k = 4$ clusters and run the
algorithm 10 times with random initialized centroids. The best result is
returned.

In [ ]:
km <- kmeans(ruspini_scaled, centers = 4, nstart = 10)
km

`km` is an R object implemented as a list. The clustering vector
contains the cluster assignment for each data row and can be accessed
using `km$cluster`. I add the cluster assignment as a column to the
scaled dataset (I make it a factor since it represents a nominal label).

In [ ]:
ruspini_clustered <- ruspini_scaled %>% add_column(cluster = factor(km$cluster))
ruspini_clustered

ggplot(ruspini_clustered, aes(x = x, y = y, color = cluster)) + geom_point()

Add the centroids to the plot.

In [ ]:
centroids <- as_tibble(km$centers, rownames = "cluster")
centroids

ggplot(ruspini_clustered, aes(x = x, y = y, color = cluster)) + geom_point() +
  geom_point(data = centroids, aes(x = x, y = y, color = cluster), shape = 3, size = 10)

Use the `factoextra` package for visualization

In [ ]:
library(factoextra)
fviz_cluster(km, data = ruspini_scaled, centroids = TRUE, repel = TRUE, ellipse.type = "norm")

#### Inspect clusters

We inspect the clusters created by the 4-cluster k-means solution. The
following code can be adapted to be used for other clustering methods.

##### Cluster Profiles

Inspect the centroids with horizontal bar charts organized by cluster.
To group the plots by cluster, we have to change the data format to the
"long"-format using a pivot operation. I use colors to match the
clusters in the scatter plots.

In [ ]:
ggplot(pivot_longer(centroids, cols = c(x, y), names_to = "feature"),
  aes(x = value, y = feature, fill = cluster)) +
  geom_bar(stat = "identity") +
  facet_grid(rows = vars(cluster))

##### Extract a single cluster

You need is to filter the rows corresponding to the cluster index. The
next example calculates summary statistics and then plots all data
points of cluster 1.

In [ ]:
cluster1 <- ruspini_clustered %>% filter(cluster == 1)
cluster1
summary(cluster1)

ggplot(cluster1, aes(x = x, y = y)) + geom_point() +
  coord_cartesian(xlim = c(-2, 2), ylim = c(-2, 2))

What happens if we try to cluster with 8 centers?

In [ ]:
fviz_cluster(kmeans(ruspini_scaled, centers = 8), data = ruspini_scaled,
  centroids = TRUE,  geom = "point", ellipse.type = "norm")

### Hierarchical Clustering

Hierarchical clustering starts with a distance matrix. `dist()` defaults
to method="Euclidean". **Note:** Distance matrices become very large
quickly (size and time complexity is $O(n^2)$ where $n$ is the number if
data points). It is only possible to calculate and store the matrix for
small data sets (maybe a few hundred thousand data points) in main
memory. If your data is too large then you can use sampling.

In [ ]:
d <- dist(ruspini_scaled)

`hclust()` implements [agglomerative hierarchical
clustering](https://en.wikipedia.org/wiki/Hierarchical_clustering). We
cluster using complete link.

In [ ]:
hc <- hclust(d, method = "complete")

Hierarchical clustering does not return cluster assignments but a
dendrogram. The standard plot function plots the dendrogram.

In [ ]:
plot(hc)

Use `factoextra` (ggplot version). We can specify the number of clusters
to visualize how the dendrogram will be cut into clusters.

In [ ]:
fviz_dend(hc, k = 4)

More plotting options for dendrograms, including plotting parts of large
dendrograms can be found [here.](https://rpubs.com/gaston/dendrograms)

Extract cluster assignments by cutting the dendrogram into four parts
and add the cluster id to the data.

In [ ]:
clusters <- cutree(hc, k = 4)
cluster_complete <- ruspini_scaled %>%
  add_column(cluster = factor(clusters))
cluster_complete

ggplot(cluster_complete, aes(x, y, color = cluster)) +
  geom_point()

Try 8 clusters (Note: `fviz_cluster` needs a list with data and the
cluster labels for hclust)

In [ ]:
fviz_cluster(list(data = ruspini_scaled, cluster = cutree(hc, k = 8)), geom = "point")

Clustering with single link

In [ ]:
hc_single <- hclust(d, method = "single")
fviz_dend(hc_single, k = 4)

fviz_cluster(list(data = ruspini_scaled, cluster = cutree(hc_single, k = 4)), geom = "point")

### Density-based clustering with DBSCAN

In [ ]:
library(dbscan)

[DBSCAN](https://en.wikipedia.org/wiki/DBSCAN) stands for "Density-Based
Spatial Clustering of Applications with Noise." It groups together
points that are closely packed together and treats points in low-density
regions as outliers.

**Parameters:** minPts defines how many points in the epsilon
neighborhood are needed to make a point a core point. It is often chosen
as a smoothing parameter. I use here minPts = 4.

To decide on epsilon, the knee in the kNN distance plot is often used.
Note that minPts contains the point itself, while the k-nearest neighbor
does not. We therefore have to use k = minPts - 1! The knee is around
eps = .32.

In [ ]:
kNNdistplot(ruspini_scaled, k = 3)
abline(h = .32, col = "red")

run dbscan

In [ ]:
db <- dbscan(ruspini_scaled, eps = .32, minPts = 4)
db
str(db)

ggplot(ruspini_scaled %>% add_column(cluster = factor(db$cluster)),
  aes(x, y, color = cluster)) + geom_point()

**Note:** Cluster 0 represents outliers).

In [ ]:
fviz_cluster(db, ruspini_scaled, geom = "point")

Play with eps (neighborhood size) and MinPts (minimum of points needed
for core cluster)

### Partitioning Around Medoids (PAM)

[PAM](https://en.wikipedia.org/wiki/K-medoids) tries to solve the
$k$-medoids problem. The problem is similar to $k$-means, but uses
medoids instead of centroids to represent clusters. Like hierarchical
clustering, it typically works with precomputed distance matrix. An
advantage is that you can use any distance metric not just Euclidean
distances. **Note:** The medoid is the most central data point in the
middle of the cluster.

In [ ]:
library(cluster)

d <- dist(ruspini_scaled)
str(d)

p <- pam(d, k = 4)
p

ruspini_clustered <- ruspini_scaled %>% add_column(cluster = factor(p$cluster))

medoids <- as_tibble(ruspini_scaled[p$medoids, ], rownames = "cluster")
medoids

ggplot(ruspini_clustered, aes(x = x, y = y, color = cluster)) + geom_point() +
  geom_point(data = medoids, aes(x = x, y = y, color = cluster), shape = 3, size = 10)

## __Note:__ `fviz_cluster` needs the original data.
fviz_cluster(c(p, list(data = ruspini_scaled)), geom = "point", ellipse.type = "norm")

### Gaussian Mixture Models

In [ ]:
library(mclust)

[Gaussian mixture
models](https://en.wikipedia.org/wiki/Mixture_model#Multivariate_Gaussian_mixture_model)
assume that the data set is the result of drawing data from a set of
Gaussian distributions where each distribution represents a cluster.
Estimation algorithms try to identify the location parameters of the
distributions and thus can be used to find clusters. `Mclust()` uses
Bayesian Information Criterion (BIC) to find the number of clusters
(model selection). BIC uses the likelihood and a penalty term to guard
against overfitting.

In [ ]:
m <- Mclust(ruspini_scaled)
summary(m)
plot(m, what = "classification")

Rerun with a fixed number of 4 clusters

In [ ]:
m <- Mclust(ruspini_scaled, G=4)
summary(m)
plot(m, what = "classification")

### Spectral clustering

[Spectral clustering](https://en.wikipedia.org/wiki/Spectral_clustering)
works by embedding the data points of the partitioning problem into the
subspace of the k largest eigenvectors of a normalized affinity/kernel
matrix. Then uses a simple clustering method like k-means.

In [ ]:
library("kernlab")

cluster_spec <- specc(as.matrix(ruspini_scaled), centers = 4)
cluster_spec

ggplot(ruspini_scaled %>% add_column(cluster = factor(cluster_spec)),
  aes(x, y, color = cluster)) + geom_point()

### Fuzzy C-Means Clustering

The [fuzzy clustering](https://en.wikipedia.org/wiki/Fuzzy_clustering)
version of the k-means clustering problem. Each data point has a degree
of membership to for each cluster.

In [ ]:
library("e1071")

cluster_cmeans <- cmeans(as.matrix(ruspini_scaled), centers = 4)
cluster_cmeans

Plot membership (shown as small pie charts)

In [ ]:
library("scatterpie")
ggplot()  +
  geom_scatterpie(data = cbind(ruspini_scaled, cluster_cmeans$membership),
    aes(x = x, y = y), cols = colnames(cluster_cmeans$membership), legend_name = "Membership") + coord_equal()

## Internal Cluster Validation

### Compare the Clustering Quality

The two most popular quality metrics are the within-cluster sum of
squares (WCSS) used by
[$k$-means](https://en.wikipedia.org/wiki/K-means_clustering) and the
[average silhouette
width](https://en.wikipedia.org/wiki/Silhouette_(clustering)). Look at
`within.cluster.ss` and `avg.silwidth` below.

In [ ]:
##library(fpc)

Notes: \* I do not load fpc since the NAMESPACE overwrites dbscan. \*
The clustering (second argument below) has to be supplied as a vector
with numbers (cluster IDs) and cannot be a factor (use `as.integer()` to
convert the factor to an ID).

In [ ]:
fpc::cluster.stats(d, km$cluster)

Read `? cluster.stats` for an explanation of all the available indices.

In [ ]:
sapply(
  list(
    km = km$cluster,
    hc_compl = cutree(hc, k = 4),
    hc_single = cutree(hc_single, k = 4)
  ),
  FUN = function(x)
    fpc::cluster.stats(d, x))[c("within.cluster.ss", "avg.silwidth"), ]

### Silhouette plot

In [ ]:
library(cluster)
plot(silhouette(km$cluster, d))

**Note:** The silhouette plot does not show correctly in R Studio if you
have too many objects (bars are missing). I will work when you open a
new plotting device with `windows()`, `x11()` or `quartz()`.

ggplot visualization using `factoextra`

In [ ]:
fviz_silhouette(silhouette(km$cluster, d))

### Find Optimal Number of Clusters for k-means

In [ ]:
ggplot(ruspini_scaled, aes(x, y)) + geom_point()

## We will use different methods and try 1-10 clusters.
set.seed(1234)
ks <- 2:10

#### Elbow Method: Within-Cluster Sum of Squares

Calculate the within-cluster sum of squares for different numbers of
clusters and look for the [knee or
elbow](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) in the
plot. (`nstart = 5` just repeats k-means 5 times and returns the best
solution)

In [ ]:
WCSS <- sapply(ks, FUN = function(k) {
  kmeans(ruspini_scaled, centers = k, nstart = 5)$tot.withinss
  })

ggplot(as_tibble(ks, WCSS), aes(ks, WCSS)) + geom_line() +
  geom_vline(xintercept = 4, color = "red", linetype = 2)

#### Average Silhouette Width

Plot the average silhouette width for different number of clusters and
look for the maximum in the plot.

In [ ]:
ASW <- sapply(ks, FUN=function(k) {
  fpc::cluster.stats(d, kmeans(ruspini_scaled, centers=k, nstart = 5)$cluster)$avg.silwidth
  })

best_k <- ks[which.max(ASW)]
best_k

ggplot(as_tibble(ks, ASW), aes(ks, ASW)) + geom_line() +
  geom_vline(xintercept = best_k, color = "red", linetype = 2)

#### Dunn Index

Use [Dunn index](https://en.wikipedia.org/wiki/Dunn_index) (another
internal measure given by min. separation/ max. diameter)

In [ ]:
DI <- sapply(ks, FUN=function(k) {
  fpc::cluster.stats(d, kmeans(ruspini_scaled, centers=k, nstart=5)$cluster)$dunn
})

best_k <- ks[which.max(DI)]
ggplot(as_tibble(ks, DI), aes(ks, DI)) + geom_line() +
  geom_vline(xintercept = best_k, color = "red", linetype = 2)

#### Gap Statistic

Compares the change in within-cluster dispersion with that expected from
a null model (see `? clusGap`). The default method is to choose the
smallest k such that its value Gap(k) is not more than 1 standard error
away from the first local maximum.

In [ ]:
library(cluster)
k <- clusGap(ruspini_scaled, FUN = kmeans,  nstart = 10, K.max = 10)
k
plot(k)

**Note:** these methods can also be used for hierarchical clustering.

There have been many other methods and indices proposed to determine the
number of clusters. See, e.g., package
[NbClust](https://cran.r-project.org/package=NbClust).

### Visualizing the Distance Matrix

In [ ]:
ggplot(ruspini_scaled, aes(x, y, color = factor(km$cluster))) + geom_point()

d <- dist(ruspini_scaled)

Inspect the distance matrix between the first 5 objects.

In [ ]:
as.matrix(d)[1:5, 1:5]

A false-color image visualizes each value in the matrix as a pixel with
the color representing the value.

In [ ]:
library(seriation)
pimage(d, col = bluered(100))

Rows and columns are the objects as they are ordered in the data set.
The diagonal represents the distance between an object and itself and
has by definition a distance of 0 (dark line). Visualizing the unordered
distance matrix does not show much structure, but we can reorder the
matrix (rows and columns) using the k-means cluster labels from cluster
1 to 4. A clear block structure representing the clusters becomes
visible.

In [ ]:
pimage(d, order=order(km$cluster), col = bluered(100))

Plot function `dissplot` in package **seriation** rearranges the matrix
and adds lines and cluster labels. In the lower half of the plot, it
shows average dissimilarities between clusters. The function organizes
the objects by cluster and then reorders clusters and objects within
clusters so that more similar objects are closer together.

In [ ]:
dissplot(d, labels = km$cluster, options=list(main="k-means with k=4"))

The reordering by `dissplot` makes the misspecification of k visible as
blocks.

In [ ]:
dissplot(d, labels = kmeans(ruspini_scaled, centers = 3)$cluster, col = bluered(100))
dissplot(d, labels = kmeans(ruspini_scaled, centers = 9)$cluster, col = bluered(100))

Using `factoextra`

In [ ]:
fviz_dist(d)

## External Cluster Validation

External cluster validation uses ground truth information. That is, the
user has an idea how the data should be grouped. This could be a known
class label not provided to the clustering algorithm.

We use an artificial data set with known groups.

In [ ]:
library(mlbench)
set.seed(1234)
shapes <- mlbench.smiley(n = 500, sd1 = 0.1, sd2 = 0.05)
plot(shapes)

Prepare data

In [ ]:
truth <- as.integer(shapes$class)
shapes <- scale(shapes$x)
colnames(shapes) <- c("x", "y")
shapes <- as_tibble(shapes)

ggplot(shapes, aes(x, y)) + geom_point()

Find optimal number of Clusters for k-means

In [ ]:
ks <- 2:20

Use within sum of squares (look for the knee)

In [ ]:
WCSS <- sapply(ks, FUN = function(k) {
  kmeans(shapes, centers = k, nstart = 10)$tot.withinss
})

ggplot(as_tibble(ks, WCSS), aes(ks, WCSS)) + geom_line()

Looks like it could be 7 clusters

In [ ]:
km <- kmeans(shapes, centers = 7, nstart = 10)

ggplot(shapes %>% add_column(cluster = factor(km$cluster)), aes(x, y, color = cluster)) +
  geom_point()

Hierarchical clustering: We use single-link because of the mouth is
non-convex and chaining may help.

In [ ]:
d <- dist(shapes)
hc <- hclust(d, method = "single")

Find optimal number of clusters

In [ ]:
ASW <- sapply(ks, FUN = function(k) {
  fpc::cluster.stats(d, cutree(hc, k))$avg.silwidth
})

ggplot(as_tibble(ks, ASW), aes(ks, ASW)) + geom_line()

The maximum is clearly at 4 clusters.

In [ ]:
hc_4 <- cutree(hc, 4)

ggplot(shapes %>% add_column(cluster = factor(hc_4)), aes(x, y, color = cluster)) +
  geom_point()

Compare with ground truth with the [corrected (=adjusted) Rand index
(ARI)](https://en.wikipedia.org/wiki/Rand_index#Adjusted_Rand_index),
the [variation of information (VI)
index](https://en.wikipedia.org/wiki/Variation_of_information),
[entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory))
and
[purity](https://en.wikipedia.org/wiki/Cluster_analysis#External_evaluation).

`cluster_stats` computes ARI and VI as comparative measures. I define
functions for entropy and purity here:

In [ ]:
entropy <- function(cluster, truth) {
  k <- max(cluster, truth)
  cluster <- factor(cluster, levels = 1:k)
  truth <- factor(truth, levels = 1:k)
  w <- table(cluster)/length(cluster)

  cnts <- sapply(split(truth, cluster), table)
  p <- sweep(cnts, 1, rowSums(cnts), "/")
  p[is.nan(p)] <- 0
  e <- -p * log(p, 2)

  sum(w * rowSums(e, na.rm = TRUE))
}

purity <- function(cluster, truth) {
  k <- max(cluster, truth)
  cluster <- factor(cluster, levels = 1:k)
  truth <- factor(truth, levels = 1:k)
  w <- table(cluster)/length(cluster)

  cnts <- sapply(split(truth, cluster), table)
  p <- sweep(cnts, 1, rowSums(cnts), "/")
  p[is.nan(p)] <- 0

  sum(w * apply(p, 1, max))
}

calculate measures (for comparison we also use random "clusterings" with
4 and 6 clusters)

In [ ]:
random_4 <- sample(1:4, nrow(shapes), replace = TRUE)
random_6 <- sample(1:6, nrow(shapes), replace = TRUE)

r <- rbind(
  kmeans_7 = c(
    unlist(fpc::cluster.stats(d, km$cluster, truth, compareonly = TRUE)),
    entropy = entropy(km$cluster, truth),
    purity = purity(km$cluster, truth)
    ),
  hc_4 = c(
    unlist(fpc::cluster.stats(d, hc_4, truth, compareonly = TRUE)),
    entropy = entropy(hc_4, truth),
    purity = purity(hc_4, truth)
    ),
  random_4 = c(
    unlist(fpc::cluster.stats(d, random_4, truth, compareonly = TRUE)),
    entropy = entropy(random_4, truth),
    purity = purity(random_4, truth)
    ),
  random_6 = c(
    unlist(fpc::cluster.stats(d, random_6, truth, compareonly = TRUE)),
    entropy = entropy(random_6, truth),
    purity = purity(random_6, truth)
    )
  )
r

Notes:

-   Hierarchical clustering found the perfect clustering.
-   Entropy and purity are heavily impacted by the number of clusters
    (more clusters improve the metric).
-   The corrected rand index shows clearly that the random clusterings
    have no relationship with the ground truth (very close to 0). This
    is a very helpful property.

Read `? cluster.stats` for an explanation of all the available indices.

## Advanced Data Preparation for Clustering

### Outlier Removal

Most clustering algorithms perform complete assignment (i.e., all data
points need to be assigned to a cluster). Outliers will affect the
clustering. It is useful to identify outliers and remove strong outliers
prior to clustering. A density based method to identify outlier is
[LOF](https://en.wikipedia.org/wiki/Local_outlier_factor) (Local Outlier
Factor). It is related to dbscan and compares the density around a point
with the densities around its neighbors (you have to specify the
neighborhood size $k$). The LOF value for a regular data point is 1. The
larger the LOF value gets, the more likely the point is an outlier.

In [ ]:
library(dbscan)

Add a clear outlier to the scaled Ruspini dataset that is 10 standard
deviations above the average for the x axis.

In [ ]:
ruspini_scaled_outlier <- ruspini_scaled %>% add_case(x=10,y=0)

#### Visual inspection of the data

Outliers can be identified using summary statistics, histograms,
scatterplots (pairs plots), and boxplots, etc. We use here a pairs plot
(the diagonal contains smoothed histograms). The outlier is visible as
the single separate point in the scatter plot and as the long tail of
the smoothed histogram for `x` (we would expect most observations to
fall in the range \[-3,3\] in normalized data).

In [ ]:
library("GGally")
ggpairs(ruspini_scaled_outlier)

The outlier is a problem for k-means

In [ ]:
km <- kmeans(ruspini_scaled_outlier, centers = 4, nstart = 10)
ruspini_scaled_outlier_km <- ruspini_scaled_outlier%>%
  add_column(cluster = factor(km$cluster))
centroids <- as_tibble(km$centers, rownames = "cluster")

ggplot(ruspini_scaled_outlier_km, aes(x = x, y = y, color = cluster)) + geom_point() +
  geom_point(data = centroids, aes(x = x, y = y, color = cluster), shape = 3, size = 10)

This problem can be fixed by increasing the number of clusters and
removing small clusters in a post-processing step or by identifying and
removing outliers before clustering.

#### Local Outlier Factor (LOF)

The [Local Outlier
Factor](https://en.wikipedia.org/wiki/Local_outlier_factor) is related
to concepts of DBSCAN can help to identify potential outliers. Calculate
the LOF (I choose a local neighborhood size of 10 for density estimation),

In [ ]:
lof <- lof(ruspini_scaled_outlier, minPts= 10)
lof

ggplot(ruspini_scaled_outlier %>% add_column(lof = lof), aes(x, y, color = lof)) +
    geom_point() + scale_color_gradient(low = "gray", high = "red")

Plot the points sorted by increasing LOF and look for a knee.

In [ ]:
ggplot(tibble(index = seq_len(length(lof)), lof = sort(lof)), aes(index, lof)) +
  geom_line() +
  geom_hline(yintercept = 1, color = "red", linetype = 2)

Choose a threshold above 1.

In [ ]:
ggplot(ruspini_scaled_outlier %>% add_column(outlier = lof >= 2), aes(x, y, color = outlier)) +
  geom_point()

Analyze the found outliers (they might be interesting data points) and then cluster the data without them.

In [ ]:
ruspini_scaled_clean <- ruspini_scaled_outlier  %>% filter(lof < 2)

km <- kmeans(ruspini_scaled_clean, centers = 4, nstart = 10)
ruspini_scaled_clean_km <- ruspini_scaled_clean%>%
  add_column(cluster = factor(km$cluster))
centroids <- as_tibble(km$centers, rownames = "cluster")

ggplot(ruspini_scaled_clean_km, aes(x = x, y = y, color = cluster)) + geom_point() +
  geom_point(data = centroids, aes(x = x, y = y, color = cluster), shape = 3, size = 10)

There are many other outlier removal strategies available. See, e.g.,
package [outliers](https://cran.r-project.org/package=outliers).

### Clustering Tendency

Most clustering algorithms will always produce a clustering, even if the
data does not contain a cluster structure. It is typically good to check
cluster tendency before attempting to cluster the data.

We use again the smiley data.

In [ ]:
library(mlbench)
shapes <- mlbench.smiley(n = 500, sd1 = 0.1, sd2 = 0.05)$x
colnames(shapes) <- c("x", "y")
shapes <- as_tibble(shapes)

#### Scatter plots

The first step is visual inspection using scatter plots.

In [ ]:
ggplot(shapes, aes(x = x, y = y)) + geom_point()

Cluster tendency is typically indicated by several separated point
clouds. Often an appropriate number of clusters can also be visually
obtained by counting the number of point clouds. We see four clusters,
but the mouth is not convex/spherical and thus will pose a problems to
algorithms like k-means.

If the data has more than two features then you can use a pairs plot
(scatterplot matrix) or look at a scatterplot of the first two principal
components using PCA. 

#### Visual Analysis for Cluster Tendency Assessment (VAT)

VAT reorders the objects to show potential clustering tendency as a
block structure (dark blocks along the main diagonal). We scale the data
before using Euclidean distance.

In [ ]:
library(seriation)

d_shapes <- dist(scale(shapes))
VAT(d_shapes, col = bluered(100))

iVAT uses the largest distances for all possible paths between two
objects instead of the direct distances to make the block structure
better visible.

In [ ]:
iVAT(d_shapes, col = bluered(100))

#### Hopkins statistic

`factoextra` can also create a VAT plot and calculate the [Hopkins
statistic](https://en.wikipedia.org/wiki/Hopkins_statistic) to assess
clustering tendency. For the Hopkins statistic, a sample of size $n$ is
drawn from the data and then compares the nearest neighbor distribution
with a simulated dataset drawn from a random uniform distribution (see
[detailed
explanation](https://www.datanovia.com/en/lessons/assessing-clustering-tendency/#statistical-methods)).
A values \>.5 indicates usually a clustering tendency.

In [ ]:
get_clust_tendency(shapes, n = 10)

Both plots show a strong cluster structure with 4 clusters.

#### Data Without Clustering Tendency

In [ ]:
data_random <- tibble(x = runif(500), y = runif(500))
ggplot(data_random, aes(x, y)) + geom_point()

No point clouds are visible, just noise.

In [ ]:
d_random <- dist(data_random)
VAT(d_random, col = bluered(100))
iVAT(d_random, col = bluered(100))
get_clust_tendency(data_random, n = 10, graph = FALSE)

There is very little clustering structure visible indicating low
clustering tendency and clustering should not be performed on this data.
However, k-means can be used to partition the data into $k$ regions of
roughly equivalent size. This can be used as a data-driven
discretization of the space.

#### k-means on Data Without Clustering Tendency

What happens if we perform k-means on data that has no inherent
clustering structure?

In [ ]:
km <- kmeans(data_random, centers = 4)
random_clustered<- data_random %>% add_column(cluster = factor(km$cluster))
ggplot(random_clustered, aes(x = x, y = y, color = cluster)) + geom_point()

k-means discretizes the space into similarly sized regions.